In [ ]:
import pandas
import pickle
import ipecharts
import pydynamo
import uncertainties
import collections
import matplotlib
import matplotlib.pyplot as plt


base = "SW_eos"
# Load the files from the fetch_data
state_data = pickle.load(open(f"{base}.raw_data.pkl", "rb"))
state_vars = set([j[0] for k in state_data.keys() for j in k])

# Turn state variables into a multi-index, with a particular preferred order
preferred_state_order = ("Lambda", "kT", "ndensity", "N")
state_vars = sorted(state_vars, key=lambda x: preferred_state_order.index(x) if x in preferred_state_order else -1)
orig_df = pickle.load(open(f"{base}.df.pkl", "rb"))
orig_df.set_index(list(state_vars), inplace=True)

# Filter out columns that do not change (boring state variables i.e.)
#orig_df = orig_df.loc[:, orig_df.nunique() > 1]

# Create a dataframe of scalars
scalar_df = orig_df.copy()
for col in scalar_df.columns:
    if scalar_df[col].dtype == "object":
        if isinstance(scalar_df[col].iloc[0], collections.abc.Iterable):
            del scalar_df[col]
scalar_df

In [ ]:
import matplotlib.pyplot as plt

for state in state_data.keys():
    if not ("Lambda", 1.5) in state or not ("kT", 1.5) in state or not ("ndensity", 1.0) in state:
        continue

    row = state_data[state]["ChungLu"]

    bond_order = row["bond_order_count"].ufloat()
    particle_order = row["order_count"].ufloat()

    curve = [(k[0] * k[1], (v / (particle_order[k[0]] * (particle_order[k[1]] - float(k[0] == k[1])))).nominal_value, (v / (particle_order[k[0]] * (particle_order[k[1]] - float(k[0] == k[1])))).std_dev) for k,v in bond_order.items()]
    print(curve)
    plt.errorbar(
        [k[0] for k in curve],
        [k[1] for k in curve],
        yerr=[k[2] for k in curve],
        label=state[-1],
        fmt="x",
    )
plt.xlabel("$k_i\\,k_j$")
plt.ylabel("$\\frac{N(k_i,\\,k_j)}{N(k_i)\\,N(k_j)}$")
plt.yscale("log")
plt.legend()
plt.grid()
plt.show()